In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
#visualisation libraries
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(style="seaborn")

import datetime
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.simplefilter("ignore")
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf

In [2]:
df_China= pd.read_csv("df_China.csv" ,encoding='unicode_escape')
df_China

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,China,2020/1/1,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000000
1,China,2020/1/2,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000000
2,China,2020/1/3,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000000
3,China,2020/1/4,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000000
4,China,2020/1/5,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,China,2021/9/24,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048299
633,China,2021/9/25,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048284
634,China,2021/9/26,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048267
635,China,2021/9/27,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048251


In [3]:
#shift 21days mortality
df_China_mortality=df_China["mortality"]
df_China_mortality.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

22     0.028169
23     0.028384
24     0.030021
25     0.027158
26     0.028641
         ...   
632    0.048299
633    0.048284
634    0.048267
635    0.048251
636    0.000000
Name: mortality, Length: 615, dtype: float64

In [4]:
#删除后21天
df_China.drop(df_China.index[616:637], inplace=True)
df_China

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,China,2020/1/1,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000000
1,China,2020/1/2,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000000
2,China,2020/1/3,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000000
3,China,2020/1/4,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000000
4,China,2020/1/5,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,China,2021/9/3,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,3.0,0.048809
612,China,2021/9/4,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,3.0,0.048795
613,China,2021/9/5,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,3.0,0.048786
614,China,2021/9/6,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,2.0,0.0,0.0,2.0,5.0,3.0,0.048767


In [5]:
df_China1=df_China.join(df_China_mortality,how="left",rsuffix="after21days")
df_China1

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality,mortalityafter21days
0,China,2020/1/1,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000
1,China,2020/1/2,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000
2,China,2020/1/3,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000
3,China,2020/1/4,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000
4,China,2020/1/5,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,China,2021/9/3,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,3.0,2.0,0.0,0.0,2.0,5.0,3.0,0.048809,0.048809
612,China,2021/9/4,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,3.0,2.0,0.0,0.0,2.0,5.0,3.0,0.048795,0.048795
613,China,2021/9/5,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,3.0,2.0,0.0,0.0,2.0,5.0,3.0,0.048786,0.048786
614,China,2021/9/6,1439323776,61%,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,3.0,2.0,0.0,0.0,2.0,5.0,3.0,0.048767,0.048767


In [6]:
df_China1.to_csv("df_China_after21days",index=False)